# Example 2: Stickleback morphometrics - landmarks

Functional morphology of organisms is often measured by placing landmarks at specific points that show structural, functional or developmental significance. In this example phenopype is used to place morphometric landmarks across the anterior half of a stickleback (*Gasterosteus aculeatus*) stained with alizarin red.

First we place landmarks in low throughput mode to learn how the landmark-function works, then we look at a high throughput landkmark example with a project directory and a global scale. 

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex2_before.jpg)
    
**Input** - Stained threespine stickleback, photographed in a glycerol bath from a camera stand
</div>
<div class="col-md-6">

![After](_assets/ex2_after.jpg)
    
**Results** - 22 landmarks are placed using the `landmark` tool from the `phenopype.measurements` module
</div>
</div>


## Low throughput

In [1]:
import phenopype as pp

filepath = r"images/stickle1.jpg"

ct = pp.load_image(filepath, cont=True) ## load image as container

In [2]:
## just place a few test-landmarks

pp.measurement.landmarks(ct, point_size=5, point_col="green", label_size=1, overwrite=True) 

## sets landmarks. the landmarks get stored inside the container. if you have already placed landmarks
## to the same container and set "overwrite=False", you wont be able to place them again until you set
## the overwrite flag to "True"

- setting landmarks


To create an control-image with the selected points, the `draw_landmarks` has to be called explicitly. This is necessary when using the low throughput, but not the high throughput routine (see [Tutorial 2](tutorial_2_phenopype_workflow.ipynb)); likewise the `save_landmarks` function.

In [3]:
pp.visualization.draw_landmarks(ct, point_size=5, point_col="green", label_size=1) 
pp.show_image(ct.canvas)
ct.df_landmarks

,filename,width,height,size_ratio_original,landmark,x,y
0,stickle1.jpg,2400,1600,1,1,1034,451
1,stickle1.jpg,2400,1600,1,2,1202,326
2,stickle1.jpg,2400,1600,1,3,1348,473


In [4]:
## draw landmarks on canvas. appearance needs to be specified here too
pp.export.save_landmarks(ct, dirpath=r"../_temp/output") ## save landmarks as csv to folder
pp.export.save_canvas(ct, dirpath=r"../_temp/output") ## also save canvas for quality control 

- landmarks saved under ../_temp/output\landmarks.csv.
- canvas saved under ../_temp/output\canvas.jpg.


###  Adding a scale

Now we will do the same thing again, but this time we use a reference image to create a scale-template so we can adjust our landmark coordinate space. This is important if for example the distance between the camera and your sample changes.  

<center>
<div style="width:500px; text-align: left">
    
![Adding a scale](_assets/ex2_scale.gif)
    
</div>
</center>

After loading the reference image, we measure the distance on the millimeter scale (click on two points inside the image), enter the distance (e.g. 10 mm), and then we create a template by dragging a rectangle around the whole (!) reference card. Finish each step with "enter".

In [5]:
import phenopype as pp

ref_path = r"images/stickleback_side.jpg"
filepath = r"images/stickle1.jpg"

ref_image = pp.load_image(ref_path) ## load image as container
ref_ratio, template_img = pp.preprocessing.create_scale(ref_image, template=True)

Scale set
- add column length
Template selected


In the next step, we load the sample image again. Then we use a classic machine learning algorithm to find the scale inside our already processed image.

In [6]:
ct = pp.load_image(filepath, cont=True) 
pp.preprocessing.find_scale(ct, template=template_img, px_mm_ratio_ref=ref_ratio, equalize=False)

---------------------------------------------------
Reference card found with 232 keypoint matches:
template image has 36 pixel per mm.
current image has 34.9 pixel per mm.
= 96.844 % of template image.
---------------------------------------------------


Now we draw the perimeter around the detected scale. Because the detected scale has coordinates like a mask we need the `draw_masks` function to draw it. 

In [7]:
pp.visualization.draw_masks(ct)

 - show mask: scale.


Now we place our landmarks again. The resulting csv now contains a column for the pixel-to-mm-ratio from the scale we detected.

In [8]:
pp.measurement.landmarks(ct, point_size=5, point_col="green", label_size=1, overwrite=True) 
pp.visualization.draw_landmarks(ct, point_size=5, point_col="green", label_size=1) 

- setting landmarks


Finally, we look at the results and export them.

In [9]:
pp.show_image(ct.canvas)
ct.df_landmarks

,filename,width,height,size_ratio_original,px_mm_ratio,landmark,x,y
0,stickle1.jpg,2400,1600,1,34.9,1,1058,521
1,stickle1.jpg,2400,1600,1,34.9,2,1132,473
2,stickle1.jpg,2400,1600,1,34.9,3,1305,456
3,stickle1.jpg,2400,1600,1,34.9,4,1512,384


In [10]:
pp.export.save_landmarks(ct, dirpath=r"../_temp/output") 
pp.export.save_canvas(ct, dirpath=r"../_temp/output")

- landmarks saved under ../_temp/output\landmarks.csv (overwritten).
- canvas saved under ../_temp/output\canvas.jpg (overwritten).


## High throughput

Now we will use the `pype` method to place landmarks - first on a single file, and then from within a Phenopype project.  

<center>
<div style="text-align: left">
    
![Landmarks in high throughput](_assets/ex2_ht.gif)
    
**High throughput method** - the `pype` method opens the image and a text editor with the pype configuration file. Any change to the configuration file, in this case the point size for landmarks, will be immediately applied to the image.  
</div>
</center>

In [16]:
import phenopype as pp

The `pype` can be used outside of a penopype project, supplying arrays or paths to images:

In [12]:
filepath = r"images/stickle1.jpg" # works
image = pp.load_image(filepath) # also works


pp.pype(filepath, name="lm1", config_preset="landmarks_1", dirpath=r"../_temp/output") 
## try supplying "image" instead of "filepath"
## dirpath specifies a directory where all results are saved



------------+++ new pype iteration 2020:03:29 18:58:32 +++--------------


AUTOLOAD
- landmarks_lm1.csv
MEASUREMENT
landmarks
- setting landmarks (overwriting)
VISUALIZATION
draw_landmarks
EXPORT
save_landmarks
- landmarks saved under ../_temp/output\landmarks_lm1.csv (overwritten).
AUTOSAVE
save_canvas
- canvas saved under ../_temp/output\canvas_lm1.jpg (overwritten).


TERMINATE


However, to increase throughput we will use the `pype` function on files organized within a phenopype project (see [Tutorial 2](tutorial_2_phenopype_workflow.ipynb) and [Tutorial 3](tutorial_3_managing_projects_1.ipynb) for more general information on high throughput workflow). 

In [13]:
myproj = pp.project(root_dir=r"../_temp/project")

myproj.add_files(image_dir="images",include="stickle", exclude=["side","top"])
myproj.add_config(name = "lm1", config_preset="landmarks_2")
myproj.add_scale(reference_image = "images/stickleback_side.jpg", template=True, overwrite=True)

pp.project.save(myproj)

--------------------------------------------
phenopype will create a new project at:

../_temp/project
Proceed? (y/n)
y
Warning - project root_dir already exists - overwrite? (y/n)y

"../_temp/project" created (overwritten)

project attributes written to ../_temp/project\attributes.yaml
--------------------------------------------
--------------------------------------------
phenopype will search for files at

images

using the following settings:

filetypes: ['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png'], include: stickle, exclude: ['side', 'top'], raw_mode: copy, search_mode: dir, unique_mode: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created

Found 3 files
--------------------------------------------
pype config generated from landmarks_2.
pype_lm1.yaml created for 0__stickle1
pype_lm1.yaml created for 0__stickle2
p

In [14]:
## if you have already run above cell you can load your project using "project.load":
myproj = pp.project.load(r"../_temp/project/project.data") ## run this if you 

After creating the project files, we can run the `pype` routine with a simple loop on `myproj.dirpaths`, which is a list of all project directories that contain the copied raw images and the config files we generated in the above cell. Interrup the loop with Esc. To resume to the point where you left, add the `skip` argument so directory with processed files are not run again.

In [15]:
for dirpath in myproj.dirpaths:
    out = pp.pype(dirpath, name="lm1", skip=True) 



------------+++ new pype iteration 2020:03:29 18:59:04 +++--------------


AUTOLOAD
- template loaded from root directory
PREPROCESSING
find_scale
---------------------------------------------------
Reference card found with 234 keypoint matches:
template image has 36 pixel per mm.
current image has 34.8 pixel per mm.
= 96.671 % of template image.
---------------------------------------------------
MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
draw_landmarks
draw_masks
 - show mask: scale.
EXPORT
save_landmarks
- landmarks saved under ../_temp/project\data\0__stickle1\landmarks_lm1.csv.
save_masks
- masks saved under ../_temp/project\data\0__stickle1\masks_lm1.csv.
AUTOSAVE
save_canvas
- canvas saved under ../_temp/project\data\0__stickle1\canvas_lm1.jpg.
save_scale
- save scale to attributes


TERMINATE


------------+++ new pype iteration 2020:03:29 18:59:10 +++--------------


AUTOLOAD
- template loaded from root directory
PREPROCESSING
find_scale
-----------------------